# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Análisis y visualización de datos (TP1 - Parte 2: Análisis detallado)

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---

## Librerías

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

## Dataset

In [ ]:
# Está la URL del archivo hosteado, pero también está el path relativo dentro de este proyecto 
# donde se escribió el data set al final del notebook `resolucion01.ipynb` 

# url = 'https://www.dropbox.com/scl/fi/684jriu2g52lqyv45z53s/ven_cln_map.csv?dl=1&rlkey=hr7lcqja46a9fj5bdx7ofdgah'
path = '../../data/interim/ven_cln_map.csv'
ventas = pd.read_csv(path, index_col=0)

In [ ]:
# Discrimanción por tipo de variables
temp_cols = ['Año', 'Mes']
ven_temp = ventas[temp_cols]

cat_cols = ['ID', 'DGR', 'Categoria', 'Trat_Fisc_Agg', 'Trat_Fisc', 'Trat_Dif',
'CM', 'Deposito', 'Modelo']
ven_cat = ventas[cat_cols]

num_cols = ['Ventas', 'Alicuota', 'Comision']
ven_num = ventas[num_cols]

---
# Análisis de variables numéricas

### Estadística descriptiva (sin considerar series temporales)

In [ ]:
ven_num.describe().T

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in num_cols:
    zScore_min = (ven_num[col].min() - ven_num[col].mean()) / ven_num[col].std()
    zScore_max = (ven_num[col].max() - ven_num[col].mean()) / ven_num[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

In [ ]:
print('Proporción de valores negativos, nulos y positivos en las variables numéricas:') 
for col in num_cols:
    print(f'\t{col}')
    ans = len(ven_num[col])
    neg = len(ven_num[ven_num[col] < 0])
    nul = len(ven_num[ven_num[col] == 0])

    cant = [neg, nul, neg+nul, ans-neg-nul, ans]
    prop = [f'{100*neg/ans:.2f}%', f'{100*nul/ans:.2f}%', f'{100*(nul+neg)/ans:.2f}%', f'{100*(ans-nul-neg)/ans:.2f}%', '100.00%']
    desc = pd.DataFrame([cant, prop], columns=['Negativos', 'Nulos', 'No positivos', 'Positivos', 'Total'], index=['Cantidad', 'Proporcion'])
    display(desc)


In [ ]:
sns.pairplot(data=ven_num, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

#### Cálculo y Visualización de la matriz de correlación entre las variables

In [ ]:
matrix = ven_num[['Ventas', 'Alicuota', 'Comision']].corr().round(2)
sns.heatmap(matrix, annot=True)

### Discriminando por ventas nulas

#### DataFrame con todas las ventas nulas

In [ ]:
ven_num_null = ven_num[ven_num['Ventas'] == 0].copy()
ven_num_null.describe().T

In [ ]:
sns.pairplot(data=ven_num_null, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

#### DataFrame con todas las ventas no nulas

In [ ]:
ven_num_not = ven_num[ven_num['Ventas'] != 0].copy()
ven_num_not.describe().T

In [ ]:
print('z-Score de los extremos de la variables numéricas:') 
for col in num_cols:
    zScore_min = (ven_num_not[col].min() - ven_num_not[col].mean()) / ven_num_not[col].std()
    zScore_max = (ven_num_not[col].max() - ven_num_not[col].mean()) / ven_num_not[col].std()

    print(f'\t{col}:')
    print(f'\t\t Mínimo: {zScore_min:.0f} \t Máximo: {zScore_max:.0f}')

In [ ]:
print('Proporción de valores negativos, nulos y positivos en las variables numéricas:') 
for col in num_cols:
    print(f'\t{col}')
    ans = len(ven_num_not[col])
    neg = len(ven_num_not[ven_num_not[col] < 0])
    nul = len(ven_num_not[ven_num_not[col] == 0])

    cant = [neg, nul, neg+nul, ans-neg-nul, ans]
    prop = [f'{100*neg/ans:.2f}%', f'{100*nul/ans:.2f}%', f'{100*(nul+neg)/ans:.2f}%', f'{100*(ans-nul-neg)/ans:.2f}%', '100.00%']
    desc = pd.DataFrame([cant, prop], columns=['Negativos', 'Nulos', 'No positivos', 'Positivos', 'Total'], index=['Cantidad', 'Proporcion'])
    display(desc)


In [ ]:
sns.pairplot(data=ven_num_not, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

### Más medidas de dispersión: asimetría y Kurtosis

In [ ]:
asim = skew(ven_num_not)
asimtest = skewtest(ven_num_not).pvalue
kurt = kurtosis(ven_num_not)
kurttest = kurtosistest(ven_num_not).pvalue

desc = pd.DataFrame({'Asimetría': asim, 'p-val Asim.': asimtest,
                     'Kurtosis (Fisher)': kurt, 'p-val Kurt.': kurttest},
                     index=['Ventas', 'Alicuota', 'Comision'])
desc

### Interpretación de la relación entre variables numéricas con etiquetas categóricas

In [ ]:
ventas_null = ventas[ventas['Ventas'] == 0].copy()
ventas_not = ventas[ventas['Ventas'] != 0].copy()

#### Ventas nulas

In [ ]:
# # Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

# for col in ['Categoria', 'Trat_Fisc_Agg', 'Trat_Fisc', 'Trat_Dif', 'CM', 'Modelo']:
#     sns.pairplot(data=ventas_null[num_cols+[col]], diag_kind='kde', hue=col, plot_kws = {'alpha': 0.3, 's': 10})
#     plt.show()

#### Ventas no nulas

In [ ]:
# # Estas gráficas tardan bastante en generarse, descomentar para obtenerlas

# for col in ['Categoria', 'Trat_Fisc_Agg', 'Trat_Fisc', 'Trat_Dif', 'CM', 'Modelo']:
#     sns.pairplot(data=ventas_not[num_cols+[col]], diag_kind='kde', hue=col, plot_kws = {'alpha': 0.3, 's': 10})
#     plt.show()

#### Ventas no nulas: más detalle con las etiquetas de `Categoria`

In [ ]:
catG1 = ['Misceláneo', 'Gondola', 'Com. Varios', 'Venta Agrop.', 'Vehiculos', 
         'Comb.', 'Farmacia']
catG2 = ['Comb. Ley', 'Supermercados', 'Veterinarios', 'Cerales', 
         'Com. Cerales', 'Mantenimiento', 'Tabaco']
catG3 = ['Comb. Reventa', 'Com. Ganado', 'Acopio Agrop.', 'Venta movil', 
         'Comunicacion', 'Gas Licuado', 'Comb. Prod.Propia']

fig, axs = plt.subplots(3, 3, figsize=(18, 15))

fig.suptitle('Ventas no nulas, clasificadas por "Categoria"')

for k in range(7):
    # alicuota vs ventas
    axs[0, 0].scatter(ventas_not[ventas_not['Categoria'] == catG1[k]]['Alicuota'], 
                    ventas_not[ventas_not['Categoria'] == catG1[k]]['Ventas'], 
                    label=catG1[k])
    axs[0, 1].scatter(ventas_not[ventas_not['Categoria'] == catG1[k]]['Alicuota'], 
                    ventas_not[ventas_not['Categoria'] == catG1[k]]['Comision'], 
                    label=catG1[k])
    axs[0, 2].scatter(ventas_not[ventas_not['Categoria'] == catG1[k]]['Comision'], 
                    ventas_not[ventas_not['Categoria'] == catG1[k]]['Ventas'], 
                    label=catG1[k])
    
    axs[1, 0].scatter(ventas_not[ventas_not['Categoria'] == catG2[k]]['Alicuota'], 
                    ventas_not[ventas['Categoria'] == catG2[k]]['Ventas'], 
                    label=catG2[k])
    axs[1, 1].scatter(ventas[ventas['Categoria'] == catG2[k]]['Alicuota'], 
                    ventas_not[ventas_not['Categoria'] == catG2[k]]['Comision'], 
                    label=catG2[k])
    axs[1, 2].scatter(ventas_not[ventas_not['Categoria'] == catG2[k]]['Comision'], 
                    ventas_not[ventas_not['Categoria'] == catG2[k]]['Ventas'], 
                    label=catG2[k])
    
    axs[2, 0].scatter(ventas_not[ventas_not['Categoria'] == catG3[k]]['Alicuota'], 
                    ventas_not[ventas_not['Categoria'] == catG3[k]]['Ventas'], 
                    label=catG3[k])
    axs[2, 1].scatter(ventas_not[ventas_not['Categoria'] == catG3[k]]['Alicuota'], 
                    ventas_not[ventas_not['Categoria'] == catG3[k]]['Comision'], 
                    label=catG3[k])
    axs[2, 2].scatter(ventas_not[ventas_not['Categoria'] == catG3[k]]['Comision'], 
                    ventas_not[ventas_not['Categoria'] == catG3[k]]['Ventas'], 
                    label=catG3[k])

axs[0, 0].legend()
axs[0, 1].legend()
axs[0, 2].legend()
axs[1, 0].legend()
axs[1, 1].legend()
axs[1, 2].legend()
axs[2, 0].legend()
axs[2, 1].legend()
axs[2, 2].legend()

axs[0, 0].set_xlabel('Alicuota [%]')
axs[1, 0].set_xlabel('Alicuota [%]')
axs[2, 0].set_xlabel('Alicuota [%]')
axs[0, 1].set_xlabel('Alicuota [%]')
axs[1, 1].set_xlabel('Alicuota [%]')
axs[2, 1].set_xlabel('Alicuota [%]')
axs[0, 2].set_xlabel('Comisión [$]')
axs[1, 2].set_xlabel('Comisión [$]')
axs[2, 2].set_xlabel('Comisión [$]')

axs[0, 0].set_ylabel('Ventas [$]')
axs[1, 0].set_ylabel('Ventas [$]')
axs[2, 0].set_ylabel('Ventas [$]')
axs[0, 1].set_ylabel('Comisión [$]')
axs[1, 1].set_ylabel('Comisión [$]')
axs[2, 1].set_ylabel('Comisión [$]')
axs[0, 2].set_ylabel('Ventas [$]')
axs[1, 2].set_ylabel('Ventas [$]')
axs[2, 2].set_ylabel('Ventas [$]')


plt.show()

#### Convenio multilateral y el vendedor 1638: generación de datasets más pequeños

In [ ]:
ventas_CM = ventas[ventas['CM'] == 1].copy()
ventas_CM['ID'].value_counts()

In [ ]:
ventas_ID = ventas[ventas['ID'] == 1638].copy()
for col in ['DGR', 'Categoria', 'Trat_Fisc_Agg', 'Trat_Fisc', 'Trat_Dif', 'Año', 'Mes', 
            'Deposito', 'Ventas', 'Alicuota', 'Comision', 'Modelo']:
    print(f'\n\n{col}')
    print(ventas_ID[col].value_counts())

In [ ]:
var = ['DGR', 'Categoria', 'Deposito', 'Comision']
val = [1638, 'Misceláneo', 154, 0.0]

for k in range(4):
    print(f'\n\n{var[k]}')
    sas = ventas[ventas[var[k]] == val[k]].copy()
    print(sas['ID'].value_counts())

In [ ]:
ventas_ID = ventas_ID.sort_values(['Año', 'Mes']).reset_index(drop=True)
ventas_ID = ventas_ID.drop(['CM'], axis=1)
print(ventas_ID.shape)

# Descomentar sólo cuando se desee guardar
# ventas_ID.to_csv('../../data/interim/vendedor1638_CM_13Cols_42Regs.csv')

In [ ]:
ventas_not = ventas_not.sort_values(['Año', 'Mes', 'Ventas']).reset_index(drop=True)
ventas_not = ventas_not.drop(['CM'], axis=1)
print(ventas_not.shape)

# Descomentar sólo cuando se desee guardar
# ventas_not.to_csv('../../data/interim/ventas_noNulas_13Cols_250448Regs.csv')

In [ ]:
ventas_null = ventas_null[ventas_null['ID'] != 1638].copy()
ventas_null = ventas_null.sort_values(['Año', 'Mes', 'Ventas']).reset_index(drop=True)
ventas_null = ventas_null.drop(['CM'], axis=1)
print(ventas_null.shape)

# Descomentar sólo cuando se desee guardar
# ventas_null.to_csv('../../data/interim/ventas_Nulas_13Cols_181016Regs.csv')

### Análisis de la relación entre variables numéricas

#### Ventas nulas

In [ ]:
# Relación lineal: comision = alicuota x venta
calcCom_null = ventas_null['Alicuota'] * ventas_null['Ventas']

L = len(calcCom_null)
S = sum(abs(ventas_null['Comision'] - calcCom_null) == 0)
print(f'{S} de {L} registros cumplen la relación >> {100*(S)/L:.2f}%')
print(f'{L-S} registros no la cumplen la relación >> {100*(L-S)/L:.2f}%')

#### Ventas no nulas

In [ ]:
# Relación lineal: comision = alicuota x venta
calcCom_not = ventas_not['Alicuota'] * ventas_not['Ventas']

diff = ventas_not['Comision'] - calcCom_not
D = len(diff)
L = len(calcCom_not)
S = sum(abs(diff) == 0)
print(f'{S} registros cumplen la relación >>> {100*(S)/L:.2f}%')
print(f'{L-S} registros no la cumplen la relación >>> {100*(L-S)/L:.2f}%')
sobre = len(diff[diff<0])
print(f'\t{sobre} registros sobreestiman la relación >>> {100*sobre/D:.2f}%')
sub = len(diff[diff>0])
print(f'\t{sub} registros subestiman la relación >>> {100*sub/D:.2f}%')

In [ ]:
com_real_sobre = ventas_not['Comision'][diff<0]
com_real_exac = ventas_not['Comision'][diff==0]
com_real_sub = ventas_not['Comision'][diff>0]

com_calc_sobre = calcCom_not[diff<0]
com_calc_exac = calcCom_not[diff==0]
com_calc_sub = calcCom_not[diff>0]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

fig.suptitle('Comisión: real vs calculada')

axs[0].scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs[0].scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs[0].scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs[0].legend()

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión calculada')

axs[1].scatter(com_real_sobre, com_calc_sobre, label='Sobreestimado', s=30)
axs[1].scatter(com_real_exac, com_calc_exac, label='Exacto', s=20)
axs[1].scatter(com_real_sub, com_calc_sub, label='Subestimado', s=10)

axs[1].legend()
axs[1].set_ylim(-5E8, 1E8)
axs[1].set_xlim(-0.5E7, 3.5E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

plt.show()

In [ ]:
rg_real = pd.concat([com_real_sub[com_calc_sub<0], com_real_sobre[com_calc_sobre>10*com_real_sobre]])
rg_calc = pd.concat([com_calc_sub[com_calc_sub<0], com_calc_sobre[com_calc_sobre>10*com_real_sobre]])

rp_real = pd.concat([com_real_sub[com_calc_sub>0], com_real_sobre[com_calc_sobre<10*com_real_sobre], com_real_exac])
rp_calc = pd.concat([com_calc_sub[com_calc_sub>0], com_calc_sobre[com_calc_sobre<10*com_real_sobre], com_calc_exac])

In [ ]:
print(f'De los {len(rp_real+rg_real)} registros:')
print(f'\t{len(rg_real)} están en la recta con pendiente grande >>> {100*len(rg_real)/len(rp_real+rg_real):.2f}%')
print(f'\t{len(rp_real)} están en la recta con pendiente pequeña >>> {100*len(rp_real)/len(rp_real+rg_real):.2f}%')

In [ ]:
LG = LR(rg_real, rg_calc)
LP = LR(rp_real, rp_calc)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[0].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[0].plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + {LG.intercept:.0f}', color='red')
axs[0].plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + {LP.intercept:.2f}', color='tab:green')

axs[0].legend(loc='upper left')

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión calculada')

axs[1].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[1].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[1].plot(rg_real, rg_real*LG.slope + LG.intercept, label=f'{LG.slope:.0f}*Real + {LG.intercept:.0f}', color='red')
axs[1].plot(rp_real, rp_real*LP.slope + LP.intercept, label=f'{LP.slope:.0f}*Real + {LP.intercept:.2f}', color='tab:green')

axs[1].legend(loc='lower right')
axs[1].set_ylim(-1E8, 1E8)
axs[1].set_xlim(-0.5E7, 4E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

fig.suptitle('Comisión empresa: real vs calculada')
plt.show()

#### Ventas no nulas, discriminando por vendedores Modelo

In [ ]:
mask0 = ventas_not['Modelo']==0
mask1 = ventas_not['Modelo']==1

In [ ]:
com_real_Mod = ventas_not['Comision'][mask1]
com_real_noMod = ventas_not['Comision'][mask0]

com_calc_Mod = calcCom_not[mask1]
com_calc_noMod = calcCom_not[mask0]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[0].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[0].plot(com_real_noMod, com_calc_noMod, label='No Modelo', color='tab:green')
axs[0].plot(com_real_Mod, com_calc_Mod, label='Modelo', color='red')

axs[0].legend(loc='upper left')

axs[0].set_xlabel('Comisión real')
axs[0].set_ylabel('Comisión Calculada')

axs[1].scatter(rg_real, rg_calc, label='Gran pendiente', s=30)
axs[1].scatter(rp_real, rp_calc, label='Pequeña pendiente', s=15)
axs[1].plot(com_real_noMod, com_calc_noMod, label='No Modelo', color='tab:green')
axs[1].plot(com_real_Mod, com_calc_Mod, label='Modelo', color='red')

axs[1].legend(loc='lower right')
axs[1].set_ylim(-1E8, 1E8)
axs[1].set_xlim(-0.5E7, 4E7)

axs[1].set_xlabel('Comisión real')
axs[1].set_ylabel('Comisión calculada')

fig.suptitle('Comisión: real vs calculada')
plt.show()

---
# Análisis de variables categóricas

#### Relación entre `Trat_Fisc` y `Trat_Fisc_Agg`

Queremos chequear para qué valores de `Trat_Fisc` aparecen valores de `Trat_Fisc_Agg`. Al analizar la tabla de contingencia, vemos que los valores de `Trat_Fisc_Agg` sólo aparecen cuando `Trat_Fisc` asume valores 0, 1, 2 ó 3. A su vez, hay correspondencia entre 0 y Normal, 1 y Exento/Desgravado, 2 y Minorista, 3 y Otro Tratamiento Fiscal. ¿Se puede unificar todo como `Trat_Fisc`, eliminando entonces `Trat_Fisc_Agg`? ¿Tiene sentido que falte la descripción asociada al tratamiento?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3 = ventas_not.copy()
prueba3['Trat_Fisc'] = prueba3['Trat_Fisc'].fillna('Waring: Relleno')
prueba3['Trat_Fisc_Agg'] = prueba3['Trat_Fisc_Agg'].fillna('Waring: Relleno')

In [ ]:
pd.crosstab(prueba3['Trat_Fisc'], prueba3['Trat_Fisc_Agg'], dropna=False)

#### Relación entre `Trat_Fisc` y `Trat_Dif`

Ahora veamos para qué valores de `Trat_Fisc` aparecen valores de `Trat_Dif`. Al analizar la tabla de contingencia vemos que no hay un patrón claro. ¿Podríamos tomar a un dato faltante como "sin tratamiento" o algo por el estilo?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3['Trat_Dif'] = prueba3['Trat_Dif'].fillna('Waring: Relleno')

In [ ]:
pd.crosstab(prueba3['Trat_Fisc'], prueba3['Trat_Dif'], dropna=False)

#### `Trat_Fisc` vs `Trat_Fisc_Agg`

Vemos que sí existe una gran correlación entre "Norm" de `Trat_Fisc_Agg` y "0" de `Trat_Fisc`.

In [ ]:
vad = pd.crosstab(ventas['Trat_Fisc'].fillna("Warning: NA"), ventas['Trat_Fisc_Agg'].fillna("Warning: NA"), normalize=True)
sns.heatmap(vad, annot=True)

#### Ventas totales por año y modelo.

In [ ]:
agg_anio = ventas.groupby(["Año", "Modelo"])["Ventas"].agg([np.mean, np.std])
agg_anio

Visualizamos el logaritmo en base 10 de las ventas, porque el valor original está muy sesgado

- Los Modelo = 1 tienen menor variablidad y mayor promedio de ventas comparados con Modelo = 0

In [ ]:
ventas_log10 = np.log(ventas[ventas["Ventas"] > 0]["Ventas"])
sns.boxplot(x=ventas["Año"], y=ventas_log10, hue=ventas["Modelo"])

#### Ventas por Descripción Tratamiento Fiscal


In [ ]:
agg_descfiscal = ventas.groupby(["Trat_Fisc_Agg", "Modelo"])['Ventas'].agg([np.mean, np.std])
agg_descfiscal

In [ ]:
sns.boxplot(data=ventas, x="Trat_Fisc_Agg", y=ventas_log10, hue="Modelo")

---
## Análisis de la serie temporal

### Construcción de una nueva variable: Fecha
Consideramos que para el análisis temporal habrá momentos en que necesitamos tener Año y Mes por separado, pero en otras ocasiones necesitamos toda la información al mismo tiempo. Por lo tanto, creamos la variable Fecha:
    $$Fecha = Año + Mes$$

In [ ]:
ventas['Fecha'] = pd.to_datetime(ventas['Mes'].astype(str) + '-' + ventas['Año'].astype(str), format='%m-%Y')
ventas.tail(3)

Vamos a considerar que la serie temporal **principal** es la dictada por `Ventas`.

### Considerando la serie temporal

#### Agrupando por Meses

Se entiende más la progresión al agrupar todos los datos correspondientes a un mismo Mes. Vemos que no hace otra cosa más que crecer. Se observa el efecto de la inflación y pareciera haber picos hacia fin de Año.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(ventas['Fecha'],ventas['Ventas'])

axs[0].set_xlabel('Fecha')
axs[0].set_ylabel('Ventas')
axs[0].set_title('Ventas sin agregar')

prueba6 = ventas[['Fecha','Ventas', 'Año']].copy()
prueba6_agregado = prueba6.groupby('Fecha')['Ventas'].sum().reset_index()

axs[1].plot(prueba6_agregado['Fecha'], prueba6_agregado['Ventas'])

axs[1].set_xlabel('Fecha')
axs[1].set_ylabel('Ventas')
axs[1].set_title('Ventas agregadas por Mes')

plt.show()

Son más similares las tendencias desde el 2019 al 2021, mientras que 2022 se destaca por ser bien diferente.

In [ ]:
Mes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

v19 = prueba6_agregado['Ventas'][:12]
v20 = prueba6_agregado['Ventas'][12:24]
v21 = prueba6_agregado['Ventas'][24:36]
v22 = prueba6_agregado['Ventas'][36:]

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(Mes, v19, label='2019', marker='o')
axs[0].plot(Mes, v20, label='2020', marker='o')
axs[0].plot(Mes, v21, label='2021', marker='o')
axs[0].plot(Mes[:6], v22, label='2022', marker='o')

axs[0].set_xlabel('Mes')
axs[0].set_ylabel('Ventas')
axs[0].set_title('Original')
axs[0].legend()

axs[1].plot(Mes, v19/np.min(v19), label='2019', marker='o')
axs[1].plot(Mes, v20/np.min(v20), label='2020', marker='o')
axs[1].plot(Mes, v21/np.min(v21), label='2021', marker='o')
axs[1].plot(Mes[:6], v22/np.min(v22), label='2022', marker='o')

axs[1].set_xlabel('Mes')
axs[1].set_ylabel('Ventas')
axs[1].set_title('Normalizadas por el menor valor del año')
axs[1].legend()

fig.suptitle('Ventas agregadas por Mes y separadas por Año')

plt.show()

#### Estacionalidad un poco más definida - mensual

Se dice que una serie de tiempo es estacionaria si no aumenta o disminuye con el tiempo de forma lineal o exponencial (sin tendencias), y si no muestra ningún tipo de patrón repetitivo (sin estacionalidad). Matemáticamente, esto se describe como una media constante y una varianza constante a lo largo del tiempo. Junto con la varianza, la autocovarianza tampoco debería ser una función del tiempo.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 10))

sns.boxplot(data=ventas, x='Mes', y='Ventas', ax=axs[0])

sns.boxplot(data=ventas, x='Mes', y='Ventas', ax=axs[1])
axs[1].set_ylim(-1E7, 1.5E7)

fig.suptitle('Estacionalidad mensual considerando los 4 Años juntos')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(24, 20))

sns.boxplot(data=ventas[ventas['Año']==2019], x='Mes', y='Ventas', ax=axs[0, 0])
sns.boxplot(data=ventas[ventas['Año']==2019], x='Mes', y='Ventas', ax=axs[1, 0])

sns.boxplot(data=ventas[ventas['Año']==2020], x='Mes', y='Ventas', ax=axs[0, 1])
sns.boxplot(data=ventas[ventas['Año']==2020], x='Mes', y='Ventas', ax=axs[1, 1])

sns.boxplot(data=ventas[ventas['Año']==2021], x='Mes', y='Ventas', ax=axs[0, 2])
sns.boxplot(data=ventas[ventas['Año']==2021], x='Mes', y='Ventas', ax=axs[1, 2])

sns.boxplot(data=ventas[ventas['Año']==2022], x='Mes', y='Ventas', ax=axs[0, 3])
sns.boxplot(data=ventas[ventas['Año']==2022], x='Mes', y='Ventas', ax=axs[1, 3])

axs[0, 0].set_title('2019')
axs[1, 0].set_ylim(-0.6E7, 1.2E7)

axs[0, 1].set_title('2020')
axs[1, 1].set_ylim(-0.6E7, 1.2E7)

axs[0, 2].set_title('2021')
axs[1, 2].set_ylim(-0.6E7, 1.2E7)

axs[0, 3].set_title('2022')
axs[1, 3].set_ylim(-0.6E7, 1.2E7)

fig.suptitle('Estacionalidad mensual por Año')
plt.show()

#### Análisis de Outliers
Van a tener influencia en las estadísticas centrales y de dispersión

**Negativos**

In [ ]:
negativos = ventas[ventas['Ventas'] < 0]
print(f'Cantidad de valores negativos:{len(negativos)}')

In [ ]:
recuento_negativos = negativos['Fecha'].value_counts().to_frame(name='count').sort_index(ascending=True)
recuento_negativos.plot()

**Ceros**

In [ ]:
ceros = ventas[ventas['Ventas'] == 0]
print(f'Cantidad de ceros: {len(ceros)}')

In [ ]:
recuento_ceros = ceros['Fecha'].value_counts().to_frame(name='count').sort_index(ascending=True)
recuento_ceros.plot()

**Extremos positivos**

In [ ]:
percentil_98 = ventas['Ventas'].quantile(0.98)
extremos = ventas[ventas['Ventas'] > percentil_98]
print(f'Cantidad de valores por encima del percentil 98: {len(extremos)}')

In [ ]:
recuento_extremos = extremos['Fecha'].value_counts().to_frame(name='count').sort_index(ascending=True)
recuento_extremos.plot()

#### Autocorrelación

Hay correlación estadísticamente significativa para los lags 1, 2 y 3, lo que implica que observaciones pasadas influyen en las observaciones actuales de la serie temporal de manera positiva.

In [ ]:
# Calcular la función de autocorrelación
acf = plot_acf(prueba6_agregado['Ventas'], lags=12)

# Graficar la función de autocorrelación
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.show()